<a href="https://colab.research.google.com/github/cagBRT/PerformanceEnhancement/blob/main/2_Pandas_Performance_Enhancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook covers some techniques for speeding up performance for large datasets (>= 1 million rows) using Pandas and Numpy

The Zen of Pandas Optimization<br>

- **Avoid loops**, if you can

- **If you must loop, use apply**, not iteration functions

- If you must apply, use Cython to make it faster

- **Vectorization is usually better than scalar operations**

- **Vector operations on NumPy arrays are more efficient than on native Pandas series**

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Intro-to-Pandas.git cloned-repo
%cd cloned-repo
!ls

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo
 adult.csv	        P2_Pandas_Data_Prep.ipynb	       testTitanic.csv
 deniro.csv	        P3_Pandas.ipynb			       titanic.csv
 letter_frequency.csv  'Pandas(1).png'			       train.csv
 oscarNoErrors.csv      Pandas_Performance_Enhancement.ipynb   trainTitanic.csv
 oscar_winners.csv      README.md			       UFO.csv
 P1_Pandas_ES.ipynb     sample_solution.csv		       xP1_Intro_to_Pandas.ipynb
 P1_Pandas.ipynb        test.csv


## Get the data

1. Download the dataset here:
###https://www.kaggle.com/competitions/tabular-playground-series-sep-2021/rules

This dataset is 1GB <br>
It has nearly 1million rows and 128 columns

2. It will be be in zip format. <br>

In [ ]:
import zipfile

Upload to the local directory:<br>   

*   tablular-playground-series-sep-2021


This can take up to 20 minutes to load

**You can continue with the notebook, this file is not needed for a little while.**



---



---



In [ ]:
import pandas as pd
import numpy as np

## Use replace to replace specific values in Pandas datasets

Speed is the first benefit of replace, <br>
the second benefit is its flexibility.

We can replace all question marks with NaN - an operation that would take multiple calls with index-based replacement.

Nested replacement helps when you only want to affect the values of specific columns. Here, we are replacing values only in education and income columns.



In [ ]:
s = pd.Series([1, 2, 3, 4, 5])
s

0    1
1    2
2    3
3    4
4    5
dtype: int64

In [ ]:
s.replace(1, 5)

0    5
1    2
2    3
3    4
4    5
dtype: int64

In [ ]:
s.replace([1, 2], method='bfill')

0    3
1    3
2    3
3    4
4    5
dtype: int64

In [ ]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})
df.replace(0, 5)

A  B  C
0  5  5  a
1  1  6  b
2  2  7  c
3  3  8  d
4  4  9  e

In [ ]:
df.replace([0, 1, 2, 3], 4)

A  B  C
0  4  5  a
1  4  6  b
2  4  7  c
3  4  8  d
4  4  9  e

In [ ]:
df.replace([0, 1, 2, 3], [4, 3, 2, 1])

A  B  C
0  4  5  a
1  3  6  b
2  2  7  c
3  1  8  d
4  4  9  e

In [ ]:
adult_income = pd.read_csv("adult.csv")

In [ ]:
adult_income.shape

(48842, 15)

In [ ]:
adult_income.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [ ]:
adult_income.isna().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [ ]:
adult_income.replace(to_replace="?", value=np.nan, inplace=True)

In [ ]:
adult_income.isna().sum()

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64

replace allows using lists or dictionaries to change multiple values simultaneously:



In [ ]:
adult_income.gender.value_counts()

Male      32650
Female    16192
Name: gender, dtype: int64

In [ ]:
adult_income.replace(["Male", "Female"], ["M", "F"], inplace=True)

In [ ]:
adult_income.gender.value_counts()

M    32650
F    16192
Name: gender, dtype: int64

When replacing a list of values with another, they will have a one-to-one, index-to-index mapping.

In [ ]:
adult_income["native-country"].value_counts()

United-States                 43832
Mexico                          951
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru                             46
Ecuador                     

In [ ]:
adult_income.replace({"United States": "USA", "US": "USA"}, inplace=True)

In [ ]:
adult_income["native-country"].value_counts()

United-States                 43832
Mexico                          951
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru                             46
Ecuador                     

In [ ]:
adult_income.replace({"United States": "USA", "US": "USA"}, inplace=True)

In [ ]:
adult_income["native-country"].value_counts()

United-States                 43832
Mexico                          951
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru                             46
Ecuador                     

In [ ]:
adult_income.education.value_counts()

HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: education, dtype: int64

In [ ]:
adult_income.income.value_counts()

<=50K    37155
>50K     11687
Name: income, dtype: int64

In [ ]:
adult_income.replace(
    {
        "education": {"HS-grad": "High school", "Some-college": "College", "12th":"High school"},
        "income": {"<=50K": 0, ">50K": 1},
    },
    inplace=True,
)

In [ ]:
adult_income.education.value_counts()

High school    16441
College        10878
Bachelors       8025
Masters         2657
Assoc-voc       2061
11th            1812
Assoc-acdm      1601
10th            1389
7th-8th          955
Prof-school      834
9th              756
Doctorate        594
5th-6th          509
1st-4th          247
Preschool         83
Name: education, dtype: int64

In [ ]:
adult_income.income.value_counts()

0    37155
1    11687
Name: income, dtype: int64

### iloc vs loc

For choosing a row or multiple rows, iloc is faster

In [ ]:
%time adult_income.iloc[range(10000)]
%time adult_income.loc[range(10000)]

CPU times: user 4.2 ms, sys: 0 ns, total: 4.2 ms
Wall time: 5.42 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.01 ms


age         workclass  fnlwgt    education  educational-num  \
0      25           Private  226802         11th                7   
1      38           Private   89814  High school                9   
2      28         Local-gov  336951   Assoc-acdm               12   
3      44           Private  160323      College               10   
4      18               NaN  103497      College               10   
...   ...               ...     ...          ...              ...   
9995   66  Self-emp-not-inc  176315    Bachelors               13   
9996   35           Private  187167  High school                9   
9997   24           Private  241582      College               10   
9998   31           Private  247328         11th                7   
9999   75  Self-emp-not-inc  157778    Assoc-voc               11   

          marital-status         occupation   relationship   race gender  \
0          Never-married  Machine-op-inspct      Own-child  Black      M   
1     Married-civ-spouse    Farming-fishing        Husband  White      M   
2     Married-civ-spouse    Protective-serv        Husband  White      M   
3     Married-civ-spouse  Machine-op-inspct        Husband  Black      M   
4          Never-married                NaN      Own-child  White      F   
...                  ...                ...            ...    ...    ...   
9995            Divorced              Sales  Not-in-family  White      M   
9996       Never-married       Adm-clerical      Unmarried  White      F   
9997       Never-married              Sales  Not-in-family  White      M   
9998  Married-civ-spouse    Protective-serv        Husband  White      M   
9999            Divorced     Prof-specialty  Not-in-family  White      F   

      capital-gain  capital-loss  hours-per-week native-country  income  
0                0             0              40  United-States       0  
1                0             0              50  United-States       0  
2                0             0              40  United-States       1  
3             7688             0              40  United-States       1  
4                0             0              30  United-States       0  
...            ...           ...             ...            ...     ...  
9995           401             0              20  United-States       0  
9996             0             0              40  United-States       0  
9997             0             0              33  United-States       0  
9998             0             0              40  United-States       0  
9999         15831             0              50  United-States       1  

[10000 rows x 15 columns]

Choosing columns:

In [ ]:
%time adult_income.loc[:,["workclass","occupation","gender"]]
%time adult_income.iloc[:,[1,6,9]]

CPU times: user 2.29 ms, sys: 0 ns, total: 2.29 ms
Wall time: 2.45 ms
CPU times: user 1.23 ms, sys: 1 µs, total: 1.23 ms
Wall time: 1.24 ms


workclass         occupation gender
0           Private  Machine-op-inspct      M
1           Private    Farming-fishing      M
2         Local-gov    Protective-serv      M
3           Private  Machine-op-inspct      M
4               NaN                NaN      F
...             ...                ...    ...
48837       Private       Tech-support      F
48838       Private  Machine-op-inspct      M
48839       Private       Adm-clerical      F
48840       Private       Adm-clerical      M
48841  Self-emp-inc    Exec-managerial      F

[48842 rows x 3 columns]

In [ ]:
%time adult_income.sample(7,axis=0)

CPU times: user 2.51 ms, sys: 920 µs, total: 3.43 ms
Wall time: 3.35 ms


age         workclass  fnlwgt    education  educational-num  \
43310   60  Self-emp-not-inc  153356  High school                9   
16841   28           Private  114053    Bachelors               13   
25316   30         State-gov  119422         10th                6   
10946   71         Local-gov   94358  High school                9   
39943   37           Private  234807  High school                9   
1057    32         State-gov  247481   Assoc-acdm               12   
6158    32           Private  352089  High school                9   

           marital-status        occupation   relationship   race gender  \
43310            Divorced             Sales  Not-in-family  Black      M   
16841       Never-married  Transport-moving  Not-in-family  White      M   
25316  Married-civ-spouse      Adm-clerical        Husband  White      M   
10946  Married-civ-spouse     Other-service        Husband  White      M   
39943            Divorced      Adm-clerical      Unmarried  White      F   
1057   Married-civ-spouse    Prof-specialty        Husband  White      M   
6158             Divorced      Adm-clerical  Not-in-family  White      F   

       capital-gain  capital-loss  hours-per-week native-country  income  
43310          2597             0              55  United-States       0  
16841             0             0              55  United-States       0  
25316             0             0              40  United-States       0  
10946             0             0               6  United-States       0  
39943             0             0              37  United-States       0  
1057              0             0              48  United-States       1  
6158              0             0              38  United-States       0

In [ ]:
%time adult_income.sample(5, axis=1).sample(7, axis=0)

CPU times: user 4.02 ms, sys: 1.84 ms, total: 5.86 ms
Wall time: 9.41 ms


education    workclass  educational-num native-country   relationship
36513    Bachelors      Private               13  United-States  Not-in-family
13324  High school      Private                9  United-States        Husband
45299      College      Private               10  United-States  Not-in-family
37442      Masters      Private               14        Germany        Husband
46025  Prof-school    State-gov               15  United-States        Husband
33403  Prof-school  Federal-gov               15  United-States           Wife
18494      5th-6th          NaN                3  United-States        Husband

### Iterating efficiently

**The golden rule for applying operations on entire columns or data frames is to never use loops**

Think about arrays as vectors and the whole data frame as a matrix

If you want to perform any mathematical operation on one or more columns, there is a good chance that the operation is vectorized in Pandas.

For example, the built-in Python operators like +, -, *, /, ** work just like on vectors.

To get a taste of vectorization, let’s perform some operations on a massive dataset. We will choose ~1M row dataset of the old Kaggle TPS September competition:

### Datatable

Datatable is a python library for manipulating tabular data.


It supports out-of-memory datasets, multi-threaded data processing, and flexible API.



The datatable module emphasizes speed and big data support (an area that pandas struggles with); it also has an expressive and concise syntax, which makes datatable also useful for small datasets.

Note: in pandas, there are two fundamental data structures: Series and DataFrame.

In [ ]:
!python3 -m pip install -U pip
!python3 -m pip install -U datatable

In [ ]:
import datatable as dt

**Only do this part if the tabular-playground-series-sssep-2021 is done uploading.**<br>

In [ ]:
!unzip "/content/tabular-playground-series-sep-2021 (3).zip"

Archive:  /content/tabular-playground-series-sep-2021 (3).zip
  inflating: sample_solution.csv     
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
tps = dt.fread("/content/cloned-repo/train.csv").to_pandas()
tps.shape

(957919, 120)

The shape of train.csv should be (957919, 120)

## Apply

The fastest built-in iterator of Pandas is apply.

Create a dataset of about 100K rows

In [ ]:
df = pd.DataFrame({
    'cola':np.random.randint(1,100, size=100000),
    'colb':np.random.randint(100,1000, size=100000),
    'colc':np.random.random(100000)
})
df.shape

(100000, 3)

Use apply on our synthetic dataset

In [ ]:
%%timeit
df['cola'].apply(lambda x: x**2)

75.5 ms ± 24.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Use a vectorized operation on our synthetic dataset

In [ ]:
%%timeit
df['cola'] ** 2

204 µs ± 21.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


For this small dataset the difference is<br>
75.5ms v 204us

Find the minimum value in each row of the dataset

Using apply

In [ ]:
%%timeit
df.apply(lambda x: x.min(), axis=1)

4.75 s ± 688 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Using vectorization

In [ ]:
%%timeit
df.min(axis=1)

1.61 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


There is a huge difference between apply and vectorization.

The reason why the difference has substantially increased is that apply function loops through each column for every row.

Using a more complex function

In [ ]:
#apply
%%timeit
df.apply(lambda x: x.max() - x.min(), axis=1)

9.03 s ± 470 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#vectorization
%%timeit
df.max(axis=1) - df.min(axis=1)

3.37 ms ± 78.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### applymap

In [ ]:
%%timeit
df.applymap(lambda x: x * 2)

101 ms ± 20 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
df * 2

429 µs ± 45.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Assignment:**<br>
Change the size of the dataset<br>
What is the largest dataset where the difference between apply and vectorization is negligible?

Apply is about 600 times faster than the fastest iterator. But we can do even better — vectorization is even faster when used on NumPy arrays:

Add .values to get the underlying NumPy ndarray of Pandas Series.

NumPy arrays are faster because they don't perform additional calls for indexing and data type

In [ ]:
def crazy_function(col1, col2, col3):
    return np.sqrt(col1 ** 3 + col2 ** 2 + col3 * 10)

Time the crazy_function on three columns using apply

In [ ]:
salary_nan_count = tps['f1'].isna().sum()
tps=tps.fillna(0)
salary_nan_count = tps['f1'].isna().sum()
print(salary_nan_count)

0


In [ ]:
%time tps['f1000'] = tps.apply(lambda row: crazy_function(row['f1'], row['f56'], row['f44']), axis=1)

<ipython-input-6-952dd1fc2005>:4: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(col1 ** 3 + col2 ** 2 + col3 * 10)


CPU times: user 16.6 s, sys: 3.94 s, total: 20.5 s
Wall time: 20.6 s


## Vectors

Watch what happens when we pass columns as vectors rather than scalars. No need to modify the function:

In [ ]:
%time tps['f1001'] = crazy_function(tps['f1'], tps['f56'], tps['f44'])

CPU times: user 37.3 ms, sys: 5.59 ms, total: 42.9 ms
Wall time: 42.1 ms


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Using apply: CPU time: 19.7 s<br>
Using vectors: CPU time: 52.8 ms<br>

Vectors much faster that using apply!

## Vectors with Numpy

In [ ]:
%time tps['f1001'] = crazy_function(tps['f2'].values, tps['f55'].values, tps['f44'].values)

CPU times: user 35.4 ms, sys: 0 ns, total: 35.4 ms
Wall time: 41.1 ms


<ipython-input-6-952dd1fc2005>:4: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(col1 ** 3 + col2 ** 2 + col3 * 10)


Pandas has a few more tricks up its sleeve.

**Fair warning, though — these won’t benefit you much unless you have upwards of +1M rows.**

In [ ]:
massive_df = pd.concat([tps.drop(["f1000", "f1001"], axis=1)] * 10)
massive_df.shape

(9579190, 120)

In [ ]:
memory_usage = massive_df.memory_usage(deep=True)
memory_usage_in_mbs = np.sum(memory_usage / 1024 ** 2)
memory_usage_in_mbs

8742.604093551636

Use our crazy_function, start with NumPy vectorization as a baseline


It takes about 0.3 seconds for a 1M row dataset

In [ ]:
%%time

massive_df["f1001"] = crazy_function(
    massive_df["f1"].values, massive_df["f56"].values, massive_df["f44"].values
)

CPU times: user 296 ms, sys: 68.8 ms, total: 365 ms
Wall time: 364 ms


<ipython-input-6-952dd1fc2005>:4: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(col1 ** 3 + col2 ** 2 + col3 * 10)


# The eval the function of Pandas

There are two versions -
> pd.eval (higher-level)

> df.eval (in the context of DataFrames).

**Like Numba, you should have at least +10,000 samples in the DataFrame to see improvements. But once you do, you will see sizeable benefits in speed.**

Let’s run our crazy_function in the context of df.eval:

In [ ]:
%%time

massive_df.eval("f1001 = (f1 ** 3 + f56 ** 2 + f44 * 10) ** 0.5", inplace=True)

CPU times: user 219 ms, sys: 78.4 ms, total: 297 ms
Wall time: 227 ms


It's not as fast as vectorization or Numba, but it has several benefits. First, you write much less code by avoiding references to the DataFrame name. Next, it significantly speeds up non-math operations on DataFrames like boolean indexing, comparisons, and many more.



**Even though we have 1-million row dataset, all our operations were under a single second.**



### **When you are not doing mathematical manipulation, evaluate your statements in pd.eval.**